## 훈련과정 : R·파이썬 기반 빅데이터 분석 전문가 양성과정
- 교과목 평가 : 머신러닝 기반 데이터 분석
- 성명 : 호지수
- 점수 :

#### Q. 아래사항을 준수하여 타이타닉 생존자 예측모델을 개발하세요.

- 데이터 : 
  - 제공 데이터 파일 : titanic3.csv
  - 훈련/검증용 데이터 : 평가 데이터 = 8 : 2
  - 훈련/검증용 데이터로 모델 학습 및 검증하고 평가 데이터는 최종 평가에만 사용


- 모델 개발 방법 :
  - 데이터 전처리 및 탐색적 분석을 통하여 파생변수 최소 2개 이상 개발
  - 알고리즘은 최소한 3개 이상 적용(Decision Tree, Random Forest, Logistic Regression은 필수)


- 훈련 및 평가 방법 :
  - GridSearchCV API를 활용하여 교차검증 및 최적 하이퍼파라미터 찾아서 학습 및 검증 수행

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# 데이터 호출
t_df = pd.read_csv('../titanic3.csv')

# 데이터 전처리
def fillna(df):
    df['age'].fillna(df['age'].mean(), inplace=True)
    df['cabin'].fillna('N', inplace=True)
    df['fare'].fillna(df['fare'].mean(), inplace=True)
    df['embarked'].fillna('N', inplace=True)
    df['body'].fillna(0, inplace=True) # 0 대신 'N'을 삽입
    return df

def drop_features(df):
    df.drop(['home.dest','boat','name','ticket'], axis=1, inplace=True)
    return df

def format_features(df):
    df['cabin'] = df['cabin'].str[:1]
    features = ['cabin','sex','embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

def set_family(df):
    df['total_family'] = df['sibsp'] + df['parch'] # total 가족 수 계산
    df.drop(['sibsp','parch'], axis=1, inplace=True) # 나뉘어진 가족 열 제외
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    df = set_family(df)
    return df

#전처리 메소드 실행
t_df = transform_features(t_df)
display(t_df.head(3))

,pclass,survived,sex,age,fare,cabin,embarked,body,total_family
0,1,1,0,29.00,211.3375,1,3,0.0,0
1,1,1,1,0.92,151.5500,2,3,0.0,3
2,1,0,0,2.00,151.5500,2,3,0.0,3


In [2]:
# 변수 분리
y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

#데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

#객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

#검증모델 호출
from sklearn.model_selection import KFold

# KFold 검증 실행
def exec_kfold(clf, folds=5):      
    kfold = KFold(n_splits=folds)  
    scores = []
    
    for iter_count, (train_index, test_index) \
    in enumerate(kfold.split(X_df)):            
        X_train, X_test = X_df.values[train_index], X_df.values[test_index]
        y_train, y_test = y_df.values[train_index], y_df.values[test_index]
        
        clf.fit(X_train, y_train)                 
        predictions = clf.predict(X_test)  
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)                      
        print('교차 검증 {0}정확도: {1:.4f}'.format(iter_count, accuracy))
        
        
    mean_score = np.mean(scores)
    print('평균 정확도: {0: .4f}'.format(mean_score))
        
# 메소드 실행(정확도 추출)
exec_kfold(dt_clf, folds=5)

교차 검증 0정확도: 0.7023
교차 검증 1정확도: 0.7634
교차 검증 2정확도: 0.7137
교차 검증 3정확도: 0.7137
교차 검증 4정확도: 0.7471
평균 정확도:  0.7281


In [3]:
# cross_val_scores
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_df, y_df, cv=10)        # cv=5는 교차검증을 얼마나 할 것인가, 숫자를 올리면 KFold와 비슷해진다.
for iter_count, accuracy in enumerate(scores):
    print('DT 교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))
    
print('DT 평균 정확도:{0:.4f}'.format(np.mean(scores)))

DT 교차 검증 0 정확도: 0.6412
DT 교차 검증 1 정확도: 0.7786
DT 교차 검증 2 정확도: 0.8473
DT 교차 검증 3 정확도: 0.7023
DT 교차 검증 4 정확도: 0.7634
DT 교차 검증 5 정확도: 0.7481
DT 교차 검증 6 정확도: 0.6336
DT 교차 검증 7 정확도: 0.6260
DT 교차 검증 8 정확도: 0.6565
DT 교차 검증 9 정확도: 0.7538
DT 평균 정확도:0.7151


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

#파라미터 생성
parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

#객체 생성
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

# 학습
grid_dclf.fit(X_train, y_train) 

print('GridSearchCV DT 최적 하이퍼 파라미터:', grid_dclf.best_params_) 
print('GridSearchCV DT 최고 정확도:', grid_dclf.best_score_)

best_dclf = grid_dclf.best_estimator_
display(best_dclf) # 출력

# DT 예측 데이터
dpredictions = best_dclf.predict(X_test) 
accuracy = accuracy_score(y_test, dpredictions)
print('Dicision Tree GSCV 예측 정확도 : ', accuracy)

GridSearchCV DT 최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV DT 최고 정확도: 0.7946365914786967


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=11, splitter='best')

Dicision Tree GSCV 예측 정확도 :  0.7938931297709924


In [5]:
#Random Forest 
rf_clf = RandomForestClassifier(random_state=11)

#메소드 실행
exec_kfold(rf_clf, folds=5)
print()

# cross_val_scores
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_clf, X_df, y_df, cv=10)
for iter_count, accuracy in enumerate(scores):
    print('RF 교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))
    
print('RF 평균 정확도:{0:.4f}'.format(np.mean(scores)))

교차 검증 0정확도: 0.7901
교차 검증 1정확도: 0.8359
교차 검증 2정확도: 0.7405
교차 검증 3정확도: 0.7061
교차 검증 4정확도: 0.7548
평균 정확도:  0.7655

RF 교차 검증 0 정확도: 0.6489
RF 교차 검증 1 정확도: 0.7939
RF 교차 검증 2 정확도: 0.8321
RF 교차 검증 3 정확도: 0.7557
RF 교차 검증 4 정확도: 0.8168
RF 교차 검증 5 정확도: 0.8015
RF 교차 검증 6 정확도: 0.6870
RF 교차 검증 7 정확도: 0.6489
RF 교차 검증 8 정확도: 0.6336
RF 교차 검증 9 정확도: 0.7154
RF 평균 정확도:0.7334


In [17]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

parameters = {'n_estimators':[50,100,200],'max_depth':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_rflf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

grid_rflf.fit(X_train, y_train) # 학습

print('GridSearchCV RF 최적 하이퍼 파라미터:', grid_rflf.best_params_) 
print('GridSearchCV RF 최고 정확도:', grid_rflf.best_score_) 

best_rflf = grid_rflf.best_estimator_
display(best_rflf)

rpredictions = best_rflf.predict(X_test)
accuracy = accuracy_score(y_test, rpredictions)
print('Random Forest GSCV 예측 정확도 : ', accuracy)

GridSearchCV RF 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 1, 'n_estimators': 200}
GridSearchCV RF 최고 정확도: 0.7965686944634314


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=11, verbose=0,
                       warm_start=False)

Random Forest GSCV 예측 정확도 :  0.7900763358778626


In [7]:
# Logistic Regression
import warnings
warnings.filterwarnings('ignore')

lr_clf = LogisticRegression(random_state=11)

exec_kfold(lr_clf, folds=5)
print()

# cross_val_scores
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr_clf, X_df, y_df, cv=10)
for iter_count, accuracy in enumerate(scores):
    print('LR 교차 검증 {0} 정확도: {1:.4f}'.format(iter_count, accuracy))
    
print('LR 평균 정확도:{0:.4f}'.format(np.mean(scores)))

교차 검증 0정확도: 0.7710
교차 검증 1정확도: 0.8550
교차 검증 2정확도: 0.7710
교차 검증 3정확도: 0.7634
교차 검증 4정확도: 0.8008
평균 정확도:  0.7922

LR 교차 검증 0 정확도: 0.6260
LR 교차 검증 1 정확도: 0.7939
LR 교차 검증 2 정확도: 0.9084
LR 교차 검증 3 정확도: 0.8550
LR 교차 검증 4 정확도: 0.8015
LR 교차 검증 5 정확도: 0.8244
LR 교차 검증 6 정확도: 0.7557
LR 교차 검증 7 정확도: 0.6794
LR 교차 검증 8 정확도: 0.7634
LR 교차 검증 9 정확도: 0.7692
LR 평균 정확도:0.7777


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

parameters = {'penalty':['l2','l1'],'C':[0.01,0.1,1,1,5,10]}

grid_lrlf = GridSearchCV(lr_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

grid_lrlf.fit(X_train, y_train) # 학습

print('GridSearchCV LR 최적 하이퍼 파라미터:', grid_lrlf.best_params_) 
print('GridSearchCV LR 최고 정확도:', grid_lrlf.best_score_) 

best_lrlf = grid_lrlf.best_estimator_
display(best_lrlf)

lpredictions = best_lrlf.predict(X_test)
accuracy = accuracy_score(y_test, lpredictions)
print('Logistic Regression GSCV 예측 정확도 :', accuracy)

GridSearchCV LR 최적 하이퍼 파라미터: {'C': 0.01, 'penalty': 'l2'}
GridSearchCV LR 최고 정확도: 0.798459785828207


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=11, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Logistic Regression GSCV 예측 정확도 : 0.8320610687022901


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

mlp_clf =MLPClassifier(random_state=10)

# MLP알고리즘의 히든레이어 노드를 10개씩 3계층으로 구성
mlp = MLPClassifier(activation='relu',\
                    alpha=0.0001,\
                    batch_size='auto',\
                    beta_1=0.9,\
                    beta_2=0.999,\
                    early_stopping=False,\
                    epsilon=1e-08,\
                    hidden_layer_sizes=(30, 30, 30),\
                    learning_rate='constant', \
                    learning_rate_init=0.001, max_iter=200,\
                    momentum=0.9, nesterovs_momentum=True,\
                    power_t=0.5, random_state=None,\
                    shuffle=True, solver='adam', \
                    tol=0.0001, validation_fraction=0.1, \
                    verbose=False, warm_start=False)

mlp.fit(X_train, y_train)

predictions = mlp.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[133  23]
 [ 32  74]]
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       156
           1       0.76      0.70      0.73       106

    accuracy                           0.79       262
   macro avg       0.78      0.78      0.78       262
weighted avg       0.79      0.79      0.79       262



In [21]:
parameters={'alpha': [1,10,0.1],'activation': ["logistic", "relu"]}


grid_mlplf = GridSearchCV(mlp_clf, param_grid=parameters, n_jobs=-1,verbose=2, scoring='accuracy', cv=5, refit=True) 
# refit=True는 최적으로 된 하이퍼 파라미터를 반영하겠다는 의미

grid_mlplf.fit(X_train, y_train) # 학습을 시킨다.

print('GridSearchCV mlp 최적 하이퍼 파라미터:', grid_mlplf.best_params_)
print('GridSearchCV mlp 최고 정확도:', grid_mlplf.best_score_)

best_mlplf = grid_mlplf.best_estimator_
mlppredictions = best_mlplf.predict(X_test)
accuracy = accuracy_score(y_test, mlppredictions)
print('dt 예측 정확도 : ', accuracy)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.6s finished


GridSearchCV mlp 최적 하이퍼 파라미터: {'activation': 'relu', 'alpha': 0.1}
GridSearchCV mlp 최고 정확도: 0.8089769879243563
dt 예측 정확도 :  0.7900763358778626


## 정리
 - Decision Tree
   - 평균 정확도:  0.7281
   - 교차검증 평균 정확도 : 0.7151
   - GridSearchCV DT 최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}
   - GridSearchCV DT 최고 정확도: 0.7946365914786967
   - Dicision Tree GSCV 예측 정확도 : 0.7938931297709924
   
 - Random Forest
   - 평균 정확도:  0.7655
   - 교차검증 평균 정확도:0.7334
   - GridSearchCV RF 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 1, 'n_estimators': 200}
   - GridSearchCV RF 최고 정확도: 0.7975210754158123
   - Random Forest GSCV 예측 정확도 :  0.7900763358778626
   
 - Logistic Regression
   - 평균 정확도:  0.7922
   - 교차검증 LR 평균 정확도:0.7777
   - GridSearchCV LR 최적 하이퍼 파라미터: {'C': 0.01, 'penalty': 'l2'}
   - GridSearchCV LR 최고 정확도: 0.798459785828207
   - Logistic Regression GSCV 예측 정확도 : 0.8320610687022901
   
 #### 가장 높은 정확도 : Logistic Regression GSCV 예측 정확도 : 83.2%